设 $\Omega=[0,1]^2$，真解$u=[x_1(1-x_1)x_2(1-x_2),0]$。$u$ 的选择确保了在边界 $\partial\Omega$ 上 $u=0$。
$$f = 
\begin{bmatrix}
35/13x_2-35/13x_2^2+10/13x_1-10/13x_1^2\\
-25/26(-1+2x_2)(-1+2x_1)
\end{bmatrix}$$
其中 $E=1$ 和 $\nu=0.3$。

In [15]:
from linear_elasticity_model2d import BoxDomainData2d
from fealpy.functionspace import LagrangeFESpace as Space

pde = BoxDomainData2d()
GD = 2
n=1
p=1
doforder = 'vdims'

mesh = pde.init_mesh(n=n)
space = Space(mesh, p=p, doforder=doforder)
uh = space.function(dim=GD)
vspace = GD*(space, )


In [16]:
from fealpy.fem import LinearElasticityOperatorIntegrator
from fealpy.fem import VectorSourceIntegrator
from fealpy.fem import VectorMassIntegrator
from fealpy.fem import BilinearForm
from fealpy.fem import LinearForm

integrator2 = VectorMassIntegrator(c=1, q=5)

bform2 = BilinearForm(vspace)
bform2.add_domain_integrator(integrator2)
MK = integrator2.assembly_cell_matrix(space=vspace)
print("MK:", MK.shape)
bform2.assembly()
M = bform2.get_matrix()
print("M:", M.shape, "\n", M.toarray().round(4))

MK: (696, 6, 6)
M: (742, 742) 
 [[0.001  0.     0.     ... 0.     0.     0.    ]
 [0.     0.001  0.     ... 0.     0.     0.    ]
 [0.     0.     0.001  ... 0.     0.     0.    ]
 ...
 [0.     0.     0.     ... 0.0011 0.     0.    ]
 [0.     0.     0.     ... 0.     0.0019 0.    ]
 [0.     0.     0.     ... 0.     0.     0.0019]]


In [17]:
import numpy as np

integrator3 = VectorSourceIntegrator(f = pde.source, q=5)

lform = LinearForm(vspace)
lform.add_domain_integrator(integrator3)
FK = integrator3.assembly_cell_vector(space = vspace)
print("FK[0]:", FK.shape)
lform.assembly()
F = lform.get_vector()
print("F:", F.shape, "\n", F.round(4))

ipoints = vspace[0].interpolation_points()
fh = pde.source(p=ipoints)
fh_1 = np.zeros(M.shape[0])
fh_1[::GD] = fh[:,0]
fh_1[1::GD] = fh[:,1]
Fh = M @ fh_1
print("Fh:", Fh.shape, "\n", Fh.round(4))


FK[0]: (696, 3, 2)
F: (742,) 
 [ 0.0002 -0.0016  0.0002  0.0016  0.0001 -0.0014  0.0001  0.0014  0.0005
 -0.0016  0.0006 -0.0011  0.0008 -0.001   0.0007 -0.0003  0.001   0.
  0.0007  0.0003  0.0008  0.001   0.0006  0.0011  0.0005  0.0016  0.0008
 -0.0013  0.0018 -0.0017  0.0015 -0.0009  0.0016 -0.0005  0.0017 -0.0002
  0.0017  0.0002  0.0016  0.0005  0.0015  0.0009  0.0018  0.0017  0.0008
  0.0013  0.0015 -0.0011  0.002  -0.0008  0.0016 -0.0004  0.0016 -0.0002
  0.0017  0.      0.0016  0.0002  0.0016  0.0004  0.002   0.0008  0.0015
  0.0011  0.0007 -0.0003  0.0025 -0.0007  0.0018 -0.0003  0.0018 -0.0002
  0.0019 -0.0001  0.0019  0.0001  0.0018  0.0002  0.0018  0.0003  0.0025
  0.0007  0.0007  0.0003  0.0019 -0.0002  0.0024 -0.0002  0.0019 -0.0001
  0.002  -0.      0.002   0.      0.002   0.      0.0019  0.0001  0.0024
  0.0002  0.0019  0.0002  0.0008  0.0001  0.0026  0.0002  0.0019  0.0001
  0.0019  0.0001  0.002   0.      0.002  -0.      0.0019 -0.0001  0.0019
 -0.0001  0.0026 -0.0002

线弹性有限元变分公式的源项那一项为$(f,\phi)$，

第一种计算方式直接利用数值积分计算，即$F=\int\phi^Tf$，

第二种方式用插值函数$\pi f$ 近似 $f$，其中 $\pi f=\phi^Tf_h$，$f_h$是离散的点，即
$$F=\int\phi^T\phi^Tf_h = Mf_h$$
其中$M$是质量矩阵。

使用一次有限元，而右端项$f$是二次函数时，此时插值误差会比较大，这是因为一次有限元的插值函数无法精确逼近$f$。这是因为一次有限元空间只能精确表示线性函数，而无法精确表示更高阶的函数，比如二次函数。

要减少这种误差，可以使用高阶有限元：使用二次或更高阶的有限元空间可以更好地逼近二次或更高阶的函数。或者细化网格：增加网格的密度可以提高结果的精度，但这会增加计算成本。

In [18]:
print("error:", np.sum(np.abs(F - Fh)))

error: 0.0021900996550663515
